In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN


In [8]:
df1 = pd.read_csv('/Users/jluisfeltrer/Desktop/Analisis de datos/data_analysis_curso/e-cmn/E-commerce_mkechinov/merged_2019_C1.csv')

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109950743 entries, 0 to 109950742
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 7.4+ GB


In [10]:
df1.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


### Primera parte: Eliminar columnas innecesarias

In [13]:
df1.drop(columns=['event_time', 'category_id', 'user_session'])

,event_type,product_id,category_code,brand,price,user_id
0,view,1003461,electronics.smartphone,xiaomi,489.07,520088904
1,view,5000088,appliances.sewing_machine,janome,293.65,530496790
2,view,17302664,NaN,creed,28.31,561587266
3,view,3601530,appliances.kitchen.washer,lg,712.87,518085591
4,view,1004775,electronics.smartphone,xiaomi,183.27,558856683
...,...,...,...,...,...,...
109950738,view,2300275,electronics.camera.video,gopro,527.40,537931532
109950739,view,10800172,NaN,redmond,61.75,527322328
109950740,view,5701038,auto.accessories.player,kenwood,128.70,566280422
109950741,view,21407424,electronics.clocks,tissot,689.85,513118352


In [14]:
# cambiar el index de la columna 'user_id a la posicion 0
df1 = df1.set_index('user_id')


In [15]:
df1.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_session
user_id,,,,,,,,
520088904,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
530496790,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,8e5f4f83-366c-4f70-860e-ca7417414283
561587266,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,755422e7-9040-477b-9bd2-6a6e8fd97387
518085591,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
558856683,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,313628f1-68b8-460d-84f6-cec7a8796ef2


In [16]:
df1.drop(columns = ['event_time', 'category_id', 'user_session'])

,event_type,product_id,category_code,brand,price
user_id,,,,,
520088904,view,1003461,electronics.smartphone,xiaomi,489.07
530496790,view,5000088,appliances.sewing_machine,janome,293.65
561587266,view,17302664,NaN,creed,28.31
518085591,view,3601530,appliances.kitchen.washer,lg,712.87
558856683,view,1004775,electronics.smartphone,xiaomi,183.27
...,...,...,...,...,...
537931532,view,2300275,electronics.camera.video,gopro,527.40
527322328,view,10800172,NaN,redmond,61.75
566280422,view,5701038,auto.accessories.player,kenwood,128.70


### Encontrar valores faltantes y/o duplicados

In [17]:
# Cuantos valores unicos hay en cada columna
df1.nunique()

event_time        5171097
event_type              3
product_id         206876
category_id           691
category_code         129
brand                4302
price               82966
user_session     23016650
dtype: int64

In [20]:
# Cuantos valores unicos hay en el índice 'user_id'
df1.index.nunique()

5316649

In [19]:
df1.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_session
user_id,,,,,,,,
520088904,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
530496790,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,8e5f4f83-366c-4f70-860e-ca7417414283
561587266,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,755422e7-9040-477b-9bd2-6a6e8fd97387
518085591,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
558856683,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,313628f1-68b8-460d-84f6-cec7a8796ef2


## Notas:
1: Durante octubre y noviembre 2019, se registraron 109950742 eventos, de 5316649 clientes unicos